RegressionMathFunctions SinXDivXMathFunction

In [1]:
%use deeplearning4j
// default backend=native. To use cuda, set backend=cuda-<version> like that:
//%use deeplearning4j(backend=cuda-10.2)
import java.util.*

In [2]:
//Random number generator seed, for reproducability
val seed = 12345

//Number of epochs (full passes of the data)
var nEpochs = 2000

//Number of data points
var nSamples = 1000

//How frequently should we plot the network output?
var plotFrequency = nEpochs/30 // => show 30 iterations

//Batch size: i.e., each epoch has nSamples/batchSize parameter updates
val batchSize = 100

//Network learning rate
val learningRate = 0.01
val rng = java.util.Random(seed.toLong())

val numInputs = 1
val numOutputs = 1


In [3]:
//var nEpochs = 1000
//var nSamples = 500
//var plotFrequency = nEpochs/30

In [4]:
interface MathFunction {
    fun getFunctionValues(x: INDArray?): INDArray?
    val name: String?
}

In [5]:
import org.nd4j.linalg.ops.transforms.Transforms

class SinXDivXMathFunction : MathFunction {
    override fun getFunctionValues(x: INDArray?): INDArray? {
        return Transforms.sin(x, true).divi(x)
    }

    override val name: String
        get() = "SinXDivX"
}

val fn: MathFunction = SinXDivXMathFunction()

In [6]:
//Generate the training data
val x = Nd4j.linspace(-10, 10, nSamples.toLong()).reshape(nSamples.toLong(), 1)
val y = fn.getFunctionValues(x)

In [7]:
%use lets-plot

In [8]:
val fn_points = geom_point( color = "green",
    data = mapOf(
        "x" to x?.data()?.asDouble()?.toList(),
        "y" to y?.data()?.asDouble()?.toList()
    ), alpha=1.0)
{
    x = "x" 
    y = "y"
}

ggplot() + fn_points

In [9]:
val list = DataSet(x, y).asList()
Collections.shuffle(list, rng)
val iterator = ListDataSetIterator(list, batchSize)

In [10]:
import org.nd4j.linalg.activations.Activation
import org.nd4j.linalg.learning.config.Nesterovs
import org.nd4j.linalg.lossfunctions.LossFunctions

val numHiddenNodes = 50

val conf = NeuralNetConfiguration.Builder()
                    .seed(seed.toLong())
                    .weightInit(WeightInit.XAVIER)
                    .updater(Nesterovs(learningRate, 0.9))
                    .list()
                    .layer(DenseLayer.Builder().nIn(numInputs).nOut(numHiddenNodes)
                            .activation(Activation.TANH).build())
                    .layer(DenseLayer.Builder().nIn(numHiddenNodes).nOut(numHiddenNodes)
                            .activation(Activation.TANH).build())
                    .layer(OutputLayer.Builder(LossFunctions.LossFunction.MSE)
                            .activation(Activation.IDENTITY)
                            .nIn(numHiddenNodes).nOut(numOutputs).build())
                    .build()
                    
//Create the network
val net = MultiLayerNetwork(conf)
net.init()
net.setListeners(ScoreIterationListener(plotFrequency*nSamples/10))                    

In [12]:
import jetbrains.letsPlot.scale.*

//Train the network on the full data set, and evaluate/plot it periodically
val networkPredictions = arrayOfNulls<INDArray>(nEpochs / plotFrequency)
var predictions_plot = ggplot()
for (i in 0 until nEpochs) {
    iterator.reset()
    net.fit(iterator)
    if ((i + 1) % plotFrequency == 0) {
        val alpha = (i+1).toDouble() / nEpochs
        val output = net.output(x, false)
        networkPredictions[i / plotFrequency] = output
        predictions_plot += geom_point( color="black",
                                data = mapOf( 
                                    "x" to x?.data()?.asDouble()?.toList(),
                                    "y" to output?.data()?.asDouble()?.toList()
                                ), alpha=alpha)
                            {
                                x = "x" 
                                y = "y"
                            } 
    }
}

predictions_plot 

In [17]:
val last_prediction_points = geom_point( color = "red",
    data = mapOf(
        "x" to x?.data()?.asDouble()?.toList(),
        "y" to networkPredictions.last()?.data()?.asDouble()?.toList()
    ), alpha=1.0)
{
    x = "x" 
    y = "y"
}

ggplot() + last_prediction_points 

In [25]:
val x_ext = Nd4j.linspace(-10, 20, nSamples.toLong()).reshape(nSamples.toLong(), 1)
val y_ext = fn.getFunctionValues(x_ext)
val output_ext = net.output(x_ext, false)

val fn_points_ext = geom_point( color = "green",
    data = mapOf(
        "x" to x_ext?.data()?.asDouble()?.toList(),
        "y" to y_ext?.data()?.asDouble()?.toList()
    ), alpha=1.0)
{
    x = "x" 
    y = "y"
}

In [26]:
val ext_prediction_points = geom_point( color = "red",
    data = mapOf(
        "x" to x_ext.data()?.asDouble()?.toList(),
        "y" to output_ext?.data()?.asDouble()?.toList()
    ), alpha=1.0)
{
    x = "x" 
    y = "y"
}

ggplot() + ext_prediction_points + fn_points_ext